In [10]:
import pandas as pd
import numpy as np
import seaborn as sns
import random
import os
from datetime import datetime
import torch
import sys
sys.path.append('../..')
from modules.many_features import utils, constants
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline

In [11]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
os.environ['PYTHONHASHSEED']=str(SEED)
torch.manual_seed(SEED)
torch.use_deterministic_algorithms(True)

In [12]:
train_df = pd.read_csv('../../final/data/train_set_noisy_6.csv')
train_df = train_df.fillna(-1)
train_df.head()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,serum_iron,rbc,gender,creatinine,cholestrol,copper,ethanol,folate,glucose,hematocrit,tsat,label
0,9.401651,-1.000000,5.840045,0.000000,116.415615,103.966653,-1.000000,2.712885,0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,28.204953,-1.000000,2.0
1,7.029873,102.717562,2.079564,-1.000000,-1.000000,79.373697,212.817785,2.657004,1,0.675576,104.688780,50.452242,59.237694,8.549773,49.832026,21.089620,-1.000000,3.0
2,15.900836,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,58.363850,-1.000000,1,0.999932,90.916733,60.845387,77.678965,1.513323,139.638756,47.702509,-1.000000,0.0
3,12.710083,437.896009,2.773303,-1.000000,392.155888,103.338649,91.089902,3.689834,1,1.431816,31.576192,124.321572,79.407308,24.911028,84.817826,38.130249,23.227983,7.0
4,9.256332,267.393719,2.347146,5.758205,258.911569,76.491274,189.160039,3.630348,1,1.259486,19.493158,-1.000000,55.681222,-1.000000,96.976233,27.768995,73.059709,3.0


In [13]:
test_df = pd.read_csv('../../final/data/test_set_constant.csv')
test_df.head()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,serum_iron,rbc,gender,creatinine,cholestrol,copper,ethanol,folate,glucose,hematocrit,tsat,label
0,7.116363,-1.000000,3.781573,2.738413,-1.000000,95.904198,68.457895,2.226085,0,1.892912,39.808550,110.329197,64.404350,21.654404,73.787009,21.349089,-1.000000,5
1,8.125320,92.230003,4.231419,1.188039,143.365567,104.057204,204.747831,2.342554,0,0.652614,13.478089,-1.000000,32.705481,-1.000000,43.520272,24.375961,142.815207,1
2,11.309450,38.324563,-1.000000,-1.000000,455.077909,76.402602,-1.000000,4.440732,0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,33.928350,-1.000000,4
3,13.763858,253.513394,2.262606,0.551444,453.772884,82.781943,90.101466,4.987993,0,0.853521,104.005514,34.639227,0.963866,22.083012,88.891838,41.291574,19.856071,0
4,11.464002,-1.000000,-1.000000,-1.000000,320.964653,104.287127,-1.000000,3.297819,0,1.163516,121.616315,105.895897,-1.000000,9.337462,-1.000000,34.392007,-1.000000,7


In [14]:
action_list = list(constants.CLASS_DICT.keys()) + [col  for col in train_df.columns if col!='label']
action_list

['No anemia',
 'Vitamin B12/Folate deficiency anemia',
 'Unspecified anemia',
 'Anemia of chronic disease',
 'Iron deficiency anemia',
 'Hemolytic anemia',
 'Aplastic anemia',
 'Inconclusive diagnosis',
 'hemoglobin',
 'ferritin',
 'ret_count',
 'segmented_neutrophils',
 'tibc',
 'mcv',
 'serum_iron',
 'rbc',
 'gender',
 'creatinine',
 'cholestrol',
 'copper',
 'ethanol',
 'folate',
 'glucose',
 'hematocrit',
 'tsat']

In [15]:
train_df.isna().sum()

hemoglobin               0
ferritin                 0
ret_count                0
segmented_neutrophils    0
tibc                     0
mcv                      0
serum_iron               0
rbc                      0
gender                   0
creatinine               0
cholestrol               0
copper                   0
ethanol                  0
folate                   0
glucose                  0
hematocrit               0
tsat                     0
label                    0
dtype: int64

In [16]:
X_train = train_df.iloc[:, 0:-1]
y_train = train_df.iloc[:, -1]

X_test = test_df.iloc[:, 0:-1]
y_test = test_df.iloc[:, -1]

X_train, y_train = np.array(X_train), np.array(y_train)
X_test, y_test = np.array(X_test), np.array(y_test)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((56000, 17), (14000, 17), (56000,), (14000,))

In [ ]:
for steps in [int(10e6), int(10.5e6), int(11e6), int(11.5e6), int(12e6), int(12.5e6), int(13e6)]:
    dqn_model = utils.stable_dqn3(X_train, y_train, steps, True, f'../../final/models/dqn_noisy_6_{steps}')

using stable baselines 3
Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 3.14     |
|    ep_rew_mean      | -0.82    |
|    exploration_rate | 0.722    |
|    success_rate     | 0.13     |
| time/               |          |
|    episodes         | 100000   |
|    fps              | 965      |
|    time_elapsed     | 303      |
|    total_timesteps  | 293112   |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 8.89     |
|    n_updates        | 60777    |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 3.4      |
|    ep_rew_mean      | -0.86    |
|    exploration_rate | 0.39     |
|    success_rate     | 0.11     |
| time/               |          |
|    episodes         | 200000   |
|    fps              | 820      |
|    t

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 3.96     |
|    ep_rew_mean      | -0.24    |
|    exploration_rate | 0.05     |
|    success_rate     | 0.45     |
| time/               |          |
|    episodes         | 1600000  |
|    fps              | 641      |
|    time_elapsed     | 10019    |
|    total_timesteps  | 6431079  |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.103    |
|    n_updates        | 1595269  |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 4.03     |
|    ep_rew_mean      | 0.04     |
|    exploration_rate | 0.05     |
|    success_rate     | 0.57     |
| time/               |          |
|    episodes         | 1700000  |
|    fps              | 639      |
|    time_elapsed     | 10708    |
|    total_timesteps  | 6852683  |
| train/              |          |
|    learning_rate  

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 5.08     |
|    ep_rew_mean      | -0.72    |
|    exploration_rate | 0.05     |
|    success_rate     | 0.2      |
| time/               |          |
|    episodes         | 700000   |
|    fps              | 469      |
|    time_elapsed     | 5635     |
|    total_timesteps  | 2647910  |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.0229   |
|    n_updates        | 649477   |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 3.9      |
|    ep_rew_mean      | -0.6     |
|    exploration_rate | 0.05     |
|    success_rate     | 0.25     |
| time/               |          |
|    episodes         | 800000   |
|    fps              | 461      |
|    time_elapsed     | 6690     |
|    total_timesteps  | 3091098  |
| train/              |          |
|    learning_rate  

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 4.13     |
|    ep_rew_mean      | -0.14    |
|    exploration_rate | 0.05     |
|    success_rate     | 0.45     |
| time/               |          |
|    episodes         | 2200000  |
|    fps              | 499      |
|    time_elapsed     | 18788    |
|    total_timesteps  | 9383922  |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.161    |
|    n_updates        | 2333480  |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 4.25     |
|    ep_rew_mean      | -0.14    |
|    exploration_rate | 0.05     |
|    success_rate     | 0.48     |
| time/               |          |
|    episodes         | 2300000  |
|    fps              | 502      |
|    time_elapsed     | 19525    |
|    total_timesteps  | 9815496  |
| train/              |          |
|    learning_rate  

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 4.93     |
|    ep_rew_mean      | -0.18    |
|    exploration_rate | 0.05     |
|    success_rate     | 0.44     |
| time/               |          |
|    episodes         | 1300000  |
|    fps              | 398      |
|    time_elapsed     | 13550    |
|    total_timesteps  | 5395502  |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.107    |
|    n_updates        | 1336375  |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 4.87     |
|    ep_rew_mean      | -0.1     |
|    exploration_rate | 0.05     |
|    success_rate     | 0.47     |
| time/               |          |
|    episodes         | 1400000  |
|    fps              | 188      |
|    time_elapsed     | 31190    |
|    total_timesteps  | 5867711  |
| train/              |          |
|    learning_rate  

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 3.79     |
|    ep_rew_mean      | -1       |
|    exploration_rate | 0.05     |
|    success_rate     | 0.05     |
| time/               |          |
|    episodes         | 400000   |
|    fps              | 625      |
|    time_elapsed     | 2252     |
|    total_timesteps  | 1407991  |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 31.9     |
|    n_updates        | 339497   |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 4.36     |
|    ep_rew_mean      | -0.94    |
|    exploration_rate | 0.05     |
|    success_rate     | 0.08     |
| time/               |          |
|    episodes         | 500000   |
|    fps              | 602      |
|    time_elapsed     | 2962     |
|    total_timesteps  | 1784943  |
| train/              |          |
|    learning_rate  

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 5        |
|    ep_rew_mean      | -0.02    |
|    exploration_rate | 0.05     |
|    success_rate     | 0.53     |
| time/               |          |
|    episodes         | 1900000  |
|    fps              | 453      |
|    time_elapsed     | 17864    |
|    total_timesteps  | 8096729  |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.0737   |
|    n_updates        | 2011682  |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 4.57     |
|    ep_rew_mean      | -0.1     |
|    exploration_rate | 0.05     |
|    success_rate     | 0.52     |
| time/               |          |
|    episodes         | 2000000  |
|    fps              | 450      |
|    time_elapsed     | 18976    |
|    total_timesteps  | 8557456  |
| train/              |          |
|    learning_rate  

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 3.95     |
|    ep_rew_mean      | -0.28    |
|    exploration_rate | 0.05     |
|    success_rate     | 0.45     |
| time/               |          |
|    episodes         | 800000   |
|    fps              | 607      |
|    time_elapsed     | 5123     |
|    total_timesteps  | 3112035  |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.0214   |
|    n_updates        | 765508   |
----------------------------------


In [ ]:
# training_env = utils.create_env(X_train, y_train)
# dqn_model = utils.load_dqn3('../../models/many_features/0.1/with_correlated_fts/dqn3_by_type_noisy_4_missing_1_7500000', training_env)
# test_df = utils.evaluate_dqn(dqn_model, X_test, y_test)
# test_df.head()

In [ ]:
#utils.diagnose_sample(dqn_model, X_test, y_test, 1)

In [ ]:
# test_df[(test_df.y_pred==1) & (test_df.y_actual==1)]

In [ ]:
test_df.y_pred.value_counts()

In [ ]:
success_rate, success_df = utils.success_rate(test_df)
success_rate

In [ ]:
avg_length, avg_return = utils.get_avg_length_reward(test_df)
avg_length, avg_return

In [ ]:
acc, f1, roc_auc = utils.test(test_df['y_actual'], test_df['y_pred'])
acc, f1, roc_auc

In [ ]:
test_df.y_pred.unique()

In [ ]:
# test_df.to_csv(f'../../test_dfs/many_features/0.1/test_df3_noisy_1_11000000.csv', index=False)
# success_df.to_csv(f'../../test_dfs/many_features/0.1/success_df3_noisy_1_11000000.csv', index=False)